# Looking at coherence 


In [1]:
# import libraries
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxa
import math

from pathlib import Path
from datetime import datetime
from time import time

import matplotlib.pyplot as plt

# uavsar code
from uavsar_pytools.convert.tiff_conversion import read_annotation

/Users/julo9057/anaconda3/envs/snowex_uavsar_coherence/lib/python3.13/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)
/Users/julo9057/anaconda3/envs/snowex_uavsar_coherence/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
import os

scripts_dir = (os.path.dirname(os.getcwd()) + '/scripts/')
sys.path.append(scripts_dir)

from plotting import plot_tifs_grid, plot_coherence_matrix
from coherence import calc_coherence_unweighted, calc_coherence_matrix

In [3]:
# load geocoded SLC data
data_dir = '../data/snowex_lowman/slcs_geocoded/'
in_dir = Path(data_dir)
slcs_05208 = sorted(list(in_dir.glob('*05208_21*geoslc.tif'))) # use just 05208 flight 2021 data for now

# create a list of geospatial xarrays 
stack = []
for slc in slcs_05208:
    print(str(slc))
    xarr = rxa.open_rasterio(str(slc))  # shape: (2, rows, cols)
    real = xarr.sel(band=1)
    imag = xarr.sel(band=2)
    complex_data = real.values + 1j * imag.values # Combine bands into complex array

    # create a new data array with complex numbers 
    complex_xarr = real.copy(data=complex_data)
    stack.append(complex_xarr)

../data/snowex_lowman/slcs_geocoded/lowman_05208_21009_005_210203_L090VV_01_BU_s2_2x8_geoslc.tif
../data/snowex_lowman/slcs_geocoded/lowman_05208_21012_004_210210_L090VV_01_BU_s2_2x8_geoslc.tif
../data/snowex_lowman/slcs_geocoded/lowman_05208_21015_009_210303_L090VV_01_BU_s2_2x8_geoslc.tif
../data/snowex_lowman/slcs_geocoded/lowman_05208_21017_019_210310_L090VV_01_BU_s2_2x8_geoslc.tif
../data/snowex_lowman/slcs_geocoded/lowman_05208_21019_019_210316_L090VV_01_BU_s2_2x8_geoslc.tif
../data/snowex_lowman/slcs_geocoded/lowman_05208_21021_007_210322_L090VV_01_BU_s2_2x8_geoslc.tif


In [ ]:
# calculate coherence for all image pairs in stack
coherences = []
for i in range(len(stack)) : 
    for j in range(len(stack)) : 
        # if (j > 2) : 
        #     break
        if (i >= j) : 
            continue
        start_time = time()
        print(f"Calculating coherence for img {i} and img {j}")
        # print(type(stack[i]))
        # print(type(stack[j]))
        coh = (calc_coherence_unweighted(stack[i], stack[j]))
        end_time = time()
        elapsed_time = end_time - start_time
        print(f"   elapsed time: {elapsed_time} seconds")

        coherences.append(coh)

# c2 = np.stack(pairs, axis=0)


Calculating coherence for img 0 and img 1


/Users/julo9057/Library/CloudStorage/OneDrive-UCB-O365/Documents/snowex_uavsar_coherence/scripts/coherence.py:36: RuntimeWarning: invalid value encountered in divide
  coherence = np.abs(numerator / denom)


   elapsed time: 29.81123185157776 seconds
Calculating coherence for img 0 and img 2
   elapsed time: 29.36965799331665 seconds
Calculating coherence for img 0 and img 3
   elapsed time: 30.520296096801758 seconds
Calculating coherence for img 0 and img 4


In [ ]:
coherence_matrix = calc_coherence_matrix(coherences, len(stack))
plot_coherence_matrix(coherence_matrix)